In [1]:
import os 
import requests
import json 
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from web.webRead import Website

In [2]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You ara able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an about page, or a company page, or carrer/Jobs page.\n"
link_system_prompt += "You should repons in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"}
        {"type": "carrers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [3]:
def get_links_user_prompt(Website):
    user_prompt = f"Here is the list of links on the website of {Website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(Website.links)
    return user_prompt

In [4]:
load_dotenv(r'C:\Users\Gabriel\Documents\key\key.env')
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found")
else:
    print("API key found")
openai = OpenAI()

MODEL = 'gpt-4o-mini'

API key found


In [5]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        
        model = MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format = {"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [6]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'},
  {'type': 'api page', 'url': 'https://www.anthropic.com/api'},
  {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

In [7]:
def get_all_details(url):
    result = "Landingpage:\n"
    result += Website(url).get_title_text()
    links = get_links(url)
    print("Found links: ", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n" 
        result += Website(link["url"]).get_title_text()
    return result

In [9]:
get_all_details("https://anthropic.com")

Found links:  {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'Claude product page', 'url': 'https://www.anthropic.com/claude'}]}


'Landingpage:\nWebpage Title:\n Home \\ Anthropic\nWebpage Contents:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and How\nMar 8, 2023\nWork with Anthropic\nAnthropic is an AI safety and research company based in San Francisco. Our interdisciplinary team has experience across ML, physics, policy, and product. Together, 

In [10]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [11]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [12]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [13]:
create_brochure("Anthropic", "https://anthropic.com")

Found links:  {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'product page', 'url': 'https://www.anthropic.com/enterprise'}]}


# Anthropic Company Brochure

Welcome to Anthropic, an AI safety and research company dedicated to creating reliable, interpretable, and steerable AI systems. Our work is more than just building technology; it's about ensuring that AI systems can be depended upon as we navigate the complexities of their impacts on society.

## Our Purpose

At Anthropic, we understand that AI will have a significant impact on the world. Our mission is to build frontier AI systems that prioritize safety, and to conduct rigorous research to understand both the opportunities and risks associated with AI. We strive to develop systems that promote not only progress but also the well-being of people and society as a whole.

## Our Products

### Claude
Meet Claude, our advanced AI model known for its intelligence and reliability. Currently featuring:
- **Claude 3.5 Sonnet** – Our most intelligent model to date.
- **Claude 3.5 Haiku** – A new addition to the Claude family.

These AI systems are designed for various applications across industries, powered by our API, and tailored to meet customer needs in enterprises, nonprofits, and more.

## AI Safety

Anthropic treats AI safety as a science. We conduct innovative research, apply our findings to our systems, and continuously engage with the broader community to share insights. Our interdisciplinary team is dedicated to frontier research in AI safety, including topics such as interpretability and reinforcement learning from human feedback.

## Culture and Values

At Anthropic, our culture is built on collaboration and trust. Here are our core values:

1. **Here for the Mission**: We focus on ensuring that transformative AI contributes positively to society. 
2. **Unusually High Trust**: We cultivate an environment where honesty and good intentions are the norm, fostering emotional maturity and intellectual openness.
3. **One Big Team**: We value collaboration across diverse teams, recognizing that our collective efforts lead to superior outcomes.

## Careers at Anthropic

Join our talented team of engineers, researchers, policy experts, and operational leaders committed to building the future of AI. We're looking for innovative thinkers who share our vision of safe and beneficial AI systems. Explore our open roles [here](#) and see how you can contribute to our mission!

## Who We Serve

Our products benefit a wide range of customers, including:
- **Businesses** seeking reliable AI solutions
- **Nonprofits** helping communities thrive
- **Educational institutions** advancing their research capabilities

## Connect With Us

Stay updated with our latest research and products by following us on our social media platforms or visiting our [website](#).

---

Become part of the changing landscape of AI with Anthropic – where safety meets innovation.